In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
random_state = 59
np.random.seed(random_state)
torch.manual_seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_state)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
dataset_path = '../data/Auto_MPG_data.csv'
dataset = pd.read_csv(dataset_path)

In [5]:
X = dataset.drop(columns='MPG').values
y = dataset['MPG'].values

In [6]:
X[:10]

array([[8.000e+00, 3.070e+02, 1.300e+02, 3.504e+03, 1.200e+01, 7.000e+01,
        0.000e+00, 0.000e+00, 1.000e+00],
       [8.000e+00, 3.500e+02, 1.650e+02, 3.693e+03, 1.150e+01, 7.000e+01,
        0.000e+00, 0.000e+00, 1.000e+00],
       [8.000e+00, 3.180e+02, 1.500e+02, 3.436e+03, 1.100e+01, 7.000e+01,
        0.000e+00, 0.000e+00, 1.000e+00],
       [8.000e+00, 3.040e+02, 1.500e+02, 3.433e+03, 1.200e+01, 7.000e+01,
        0.000e+00, 0.000e+00, 1.000e+00],
       [8.000e+00, 3.020e+02, 1.400e+02, 3.449e+03, 1.050e+01, 7.000e+01,
        0.000e+00, 0.000e+00, 1.000e+00],
       [8.000e+00, 4.290e+02, 1.980e+02, 4.341e+03, 1.000e+01, 7.000e+01,
        0.000e+00, 0.000e+00, 1.000e+00],
       [8.000e+00, 4.540e+02, 2.200e+02, 4.354e+03, 9.000e+00, 7.000e+01,
        0.000e+00, 0.000e+00, 1.000e+00],
       [8.000e+00, 4.400e+02, 2.150e+02, 4.312e+03, 8.500e+00, 7.000e+01,
        0.000e+00, 0.000e+00, 1.000e+00],
       [8.000e+00, 4.550e+02, 2.250e+02, 4.425e+03, 1.000e+01, 7.000e+01

In [7]:
val_size = 0.2
test_size = 0.125
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=val_size, 
    random_state=random_state, 
    shuffle=is_shuffle
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=test_size, 
    random_state=random_state, 
    shuffle=is_shuffle
)

In [8]:
# Chuẩn hóa đặc trưng đầu vào
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

#### Xây dựng DataLoader


In [9]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [10]:
batch_size = 32
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)
val_loader = DataLoader(val_dataset, 
                        batch_size=batch_size, 
                        shuffle=False)

#### Xây dựng mạng MLP

In [11]:
# Bài 8.1: chuyển kiến trúc class MLP về dạng Linear Regression
class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super().__init__()
        # self.linear1 = nn.Linear(input_dims, hidden_dims)
        # self.linear2 = nn.Linear(hidden_dims, hidden_dims)
        self.output = nn.Linear(input_dims, output_dims)

    def forward(self, x):
        # x = self.linear1(x)
        # x = self.linear2(x)
        out = self.output(x)
        return out.squeeze(1)

In [12]:
input_dims = X_train.shape[1]
output_dims = 1
hidden_dims = 64

model = MLP(input_dims=input_dims, 
            hidden_dims=hidden_dims, 
            output_dims=output_dims).to(device)

#### Khai báo hàm loss và optimizer


In [13]:
lr = 1e-2
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

#### Xây dựng hàm tính điểm R2

In [14]:
def r_squared(y_true, y_pred):
    y_true = torch.tensor(y_true).to(device)
    y_pred = torch.tensor(y_pred).to(device)
    mean_true = torch.mean(y_true)
    ss_tot = torch.sum((y_true - mean_true) ** 2)
    ss_res = torch.sum((y_true - y_pred) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    return r2

#### Huấn luyện mô hình


In [15]:
epochs = 100
train_losses = []
val_losses = []
train_r2 = []
val_r2 = []

for epoch in range(epochs):
    train_loss = 0.0
    train_target = []
    val_target = []
    train_predict = []
    val_predict = []
    model.train()
    for X_samples, y_samples in train_loader:
        X_samples = X_samples.to(device)
        y_samples = y_samples.to(device)

        optimizer.zero_grad()
        outputs = model(X_samples)
        train_predict += outputs.tolist()
        train_target += y_samples.tolist()
        loss = criterion(outputs, y_samples)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    train_r2.append(r_squared(train_target, train_predict))

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_samples, y_samples in val_loader:
            X_samples = X_samples.to(device)
            y_samples = y_samples.to(device)
            outputs = model(X_samples)
            val_predict += outputs.tolist()
            val_target += y_samples.tolist()
            loss = criterion(outputs, y_samples)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    
    val_r2.append(r_squared(val_target, val_predict))
    print(f'Epoch {epoch+1}:\tTraining Loss: {train_loss:.3f}\t Validation Loss: {val_loss:.3f}')

Epoch 1:	Training Loss: 476.805	 Validation Loss: 385.831
Epoch 2:	Training Loss: 303.106	 Validation Loss: 254.122
Epoch 3:	Training Loss: 196.608	 Validation Loss: 169.873
Epoch 4:	Training Loss: 129.926	 Validation Loss: 115.707
Epoch 5:	Training Loss: 87.864	 Validation Loss: 80.139
Epoch 6:	Training Loss: 61.146	 Validation Loss: 55.858
Epoch 7:	Training Loss: 43.555	 Validation Loss: 40.130
Epoch 8:	Training Loss: 32.879	 Validation Loss: 30.218
Epoch 9:	Training Loss: 25.408	 Validation Loss: 23.337
Epoch 10:	Training Loss: 21.030	 Validation Loss: 18.796
Epoch 11:	Training Loss: 17.650	 Validation Loss: 15.728
Epoch 12:	Training Loss: 15.879	 Validation Loss: 13.729
Epoch 13:	Training Loss: 14.643	 Validation Loss: 12.188
Epoch 14:	Training Loss: 13.732	 Validation Loss: 11.165
Epoch 15:	Training Loss: 13.098	 Validation Loss: 10.592
Epoch 16:	Training Loss: 13.014	 Validation Loss: 9.990
Epoch 17:	Training Loss: 12.674	 Validation Loss: 9.644
Epoch 18:	Training Loss: 12.373	 V

#### Đánh giá mô hình


In [16]:
model.eval()
with torch.no_grad():
    y_hat = model(X_test.to(device))
    test_set_r2 = r_squared(y_test, y_hat)
    print('Evaluation on test set:')
    print(f'R2: {test_set_r2}')

Evaluation on test set:
R2: 0.8678443431854248


C:\Users\FPTSHOP\AppData\Local\Temp\ipykernel_20884\1949884570.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_true = torch.tensor(y_true).to(device)
C:\Users\FPTSHOP\AppData\Local\Temp\ipykernel_20884\1949884570.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = torch.tensor(y_pred).to(device)
